In [93]:
import pandas as pd
import numpy as np

In [94]:
calendario = pd.read_excel('../dataset/calendario^(Stagione 2022_23).xlsx')

In [95]:
calendario['HomeTeam'] = calendario.Partita.apply(lambda x: str(x).replace(' ','').split('-')[0].upper()[0:3])
calendario['AwayTeam'] = calendario.Partita.apply(lambda x: str(x).replace(' ','').split('-')[1].upper()[0:3])

In [96]:
dixon_coles = pd.read_csv('dixon_coles^(Stagione 2021_22).csv',index_col=[0])

In [97]:
new_team_remap = { 'GEN' : 'CRE', 'CAG':'MON', 'VEN':'LEC' }

In [98]:
dixon_coles['Squadra'] = dixon_coles.index.tolist()
for idx in dixon_coles.index:
    if idx in new_team_remap.keys():
        dixon_coles.loc[idx,'Squadra'] = new_team_remap[idx]
dixon_coles = dixon_coles.set_index('Squadra')

In [123]:
dixon_coles

,Atk_Params,Def_Params
Squadra,,
ATA,1.237182,-0.864092
BOL,0.854800,-0.749950
MON,0.606601,-0.548694
EMP,0.997852,-0.505108
FIO,1.138049,-0.819373
CRE,0.373825,-0.676658
INT,1.479094,-1.254770
JUV,1.091228,-1.139555
LAZ,1.417609,-0.666181


In [99]:
calendario = calendario.merge(dixon_coles, left_on='HomeTeam',right_index=True).rename(columns={'Atk_Params':'HomeTeam_atk','Def_Params':'HomeTeam_def'})
calendario = calendario.merge(dixon_coles, left_on='AwayTeam',right_index=True).rename(columns={'Atk_Params':'AwayTeam_atk','Def_Params':'AwayTeam_def'})

In [100]:
calendario['atk_home'] = calendario['HomeTeam_atk'] + calendario['AwayTeam_def']
calendario['atk_away'] = calendario['AwayTeam_atk'] + calendario['AwayTeam_def']

In [101]:
home_atk_giornate = calendario.groupby(['Giornata','HomeTeam'])['atk_home'].mean()
away_atk_giornate = calendario.groupby(['Giornata','AwayTeam'])['atk_away'].mean()

In [119]:
calendario_explode = pd.concat([home_atk_giornate, away_atk_giornate]).reset_index().rename(columns={0:'atk','HomeTeam':'Squadra'})

In [120]:
atk_pivoted = calendario_explode.pivot('Giornata','Squadra','atk')